In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, UnidentifiedImageError

# 1. Check if we are running in the Kaggle environment
in_kaggle = os.path.exists('/kaggle/working')
if in_kaggle:
    print("Running in Kaggle environment.")
else:
    print("Not running in Kaggle environment. Adjust paths if necessary.")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [ ]:
%cd /kaggle/working
!git clone https://github.com/MMatviiuk/Task_2_Image_Classification.git
!mv Task_NER_Image_Classification/* .  
!rm -rf Task_2_Image_Classification  
!ls -R  

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
!jupyter nbconvert --to notebook --execute --inplace EDA.ipynb
%run EDA.ipynb

In [ ]:
!rm -rf /kaggle/working/ner_model_bert/*
!rm -rf /kaggle/working/ner_model_roberta/*
!rm -rf /kaggle/working/__pycache__/
!rm -rf /kaggle/working/*.pth


In [ ]:
!python train_ner_bert.py \
    --train_file ./ner_data/ner_train.json \
    --val_file ./ner_data/ner_val.json \
    --model_name bert-base-cased \
    --epochs 10 \
    --batch_size 16 \
    --lr 3e-5 \
    --max_length 64 \
    --output_dir ./ner_model_bert

In [ ]:
!rm -rf /kaggle/working/ner_model_roberta/*
!rm -rf /kaggle/working/__pycache__/
!rm -rf /kaggle/working/*.pth


In [ ]:
!python train_ner_roberta.py \
    --train_file ./ner_data/ner_train.json \
    --val_file ./ner_data/ner_val.json \
    --model_name roberta-base \
    --epochs 10 \
    --batch_size 16 \
    --lr 3e-5 \
    --max_length 64 \
    --output_dir ./ner_model_roberta

In [ ]:
!python infer_ner_ensemble.py \
    --text "I saw a cat near the lake." \
    --bert_model_dir ./ner_model_bert \
    --roberta_model_dir ./ner_model_roberta


In [ ]:
!python train_image_classifier.py \
    --data_dir /kaggle/input/animals10/raw-img/ \
    --epochs 10 \
    --batch_size 32 \
    --lr 0.001 \
    --output_model ./image_classifier.pth

In [ ]:
!python infer_ner_ensemble.py \
    --text "There is an elephant resting in the grass." \
    --bert_model_dir ./ner_model_bert \
    --roberta_model_dir ./ner_model_roberta


In [ ]:
!python pipeline_ensemble.py \
    --text "There is an elephant in the picture." \
    --image_path /kaggle/input/animals10/raw-img/elefante/OIP--NEqn4JVnn251xGu7ss4bQHaHa.jpeg \
    --bert_model_dir ./ner_model_bert \
    --roberta_model_dir ./ner_model_roberta \
    --img_model ./image_classifier.pth \
    --num_classes 10
